In [33]:
#!pip install webdriver-manager --upgrade
#!pip install packaging
#!pip install -U selenium==4.11.2

  Attempting uninstall: webdriver-manager
    Found existing installation: webdriver-manager 3.8.5
    Uninstalling webdriver-manager-3.8.5:
      Successfully uninstalled webdriver-manager-3.8.5


In [12]:
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

driver = webdriver.Chrome()
driver.get("https://www.youtube.com/@betterideas/videos")

scroll_pause_time = 2  # Reduce the pause time for faster scrolling
scroll_increment = 3  # Number of scroll steps in one go

# Scroll to the bottom of the page
while True:
    for _ in range(scroll_increment):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)  # Short pause between each scroll step

    time.sleep(scroll_pause_time)
    
    new_height = driver.execute_script('return document.documentElement.scrollHeight')
    
    if new_height == last_height:
        break
    
    last_height = new_height

titles = driver.find_elements(By.ID, "video-title")
images = driver.find_elements(By.XPATH, '//a[@id="thumbnail"]/yt-image/img')
data = []

for i, j in zip(titles, images):
    data.append([i.text, j.get_attribute('src')])

df = pd.DataFrame(data, columns=['title', 'thumbnail'])

driver.quit()


In [13]:
df

,title,thumbnail
0,8 steps to unf*** your life,https://i.ytimg.com/vi/FkQ4iWloT_s/hqdefault.j...
1,Having no charisma is ruining your life,https://i.ytimg.com/vi/hzuMdEj7aO0/hqdefault.j...
2,How to have the best summer of your life,https://i.ytimg.com/vi/1Mc5P6NJzAI/hqdefault.j...
3,How optimizing my sleep is making me limitless,https://i.ytimg.com/vi/-fY6J97K67E/hqdefault.j...
4,The only way to stop being broke,https://i.ytimg.com/vi/SpHDX98K41k/hqdefault.j...
5,I can't keep doing this to myself,https://i.ytimg.com/vi/0_MEqKi_i7Y/hqdefault.j...
6,8 movies that will quickly improve your life,https://i.ytimg.com/vi/fzNMm6BE1GM/hqdefault.j...
7,The secret to effortless improvement,https://i.ytimg.com/vi/Rn2-eiuZGL4/hqdefault.j...
8,Escape the cave,https://i.ytimg.com/vi/eoDiqFyavdA/hqdefault.j...
9,The biggest habit building mistake,https://i.ytimg.com/vi/B-d79DtMTiE/hqdefault.j...


In [14]:
count = df['thumbnail'].isna().sum()
print(f'count none values:{count}')

count none values:0


In [17]:
cell_value = df.at[9, 'thumbnail']
print("Contents of cell:", cell_value)

Contents of cell: https://i.ytimg.com/vi/B-d79DtMTiE/hqdefault.jpg?sqp=-oaymwEcCNACELwBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDm9wpB5_lziXus1NBQ4_6Xjl6J8A


In [19]:
import os
import re
import requests

# Create a folder to save the thumbnails
if not os.path.exists('bi_thumbnails'):
    os.mkdir('bi_thumbnails')

# Assuming you already have a DataFrame named 'df' with 'title' and 'thumbnail' columns
for index, row in df.iterrows():
    title = row['title']
    thumbnail_url = row['thumbnail']
    
    # Replace invalid characters in the title to create a valid filename
    filename = re.sub(r'[\/:*?"<>|]', '', title) + '.jpg'
    filepath = os.path.join('bi_thumbnails', filename)
    
    # Download the thumbnail
    response = requests.get(thumbnail_url)
    if response.status_code == 200:
        with open(filepath, 'wb') as file:
            file.write(response.content)
